# スリザーリンクデータセット生成
スリザーリンクパズルをDNNに学習させるためのデータセットを生成します。

# Step1. パズルデータ取得
pinkston3さんのslitherlinkリポジトリをforkして改変した[pinkston3_slitherlink](https://github.com/s-n-1-0/pinkston3_slitherlink)リポジトリのget_puzzle.pyを実行します。
※500件のパズルを取得可能(save.npzに保存)

# Step2 . 正解データ生成
SaitoTsutomuさんの[opt4puzzleリポジトリ](https://github.com/SaitoTsutomu/opt4puzzle)のスリザーリンクパズル解法を元に生成して保存します。
+ [Qiita](https://qiita.com/SaitoTsutomu/items/0c0db8d22979fc9de8f4)
+ コード出典:[SaitoTsutomu/opt4puzzleリポジトリ SlitherLink.ipyn](https://github.com/SaitoTsutomu/opt4puzzle/blob/master/SlitherLink.ipynb)

In [1]:
import numpy as np
from itertools import product
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars

In [2]:
data = np.load("./save.npz")['arr_0']
print(data.shape)

(500, 10, 10)


In [18]:
print("\n".join(["".join(r) for r in data[0]]).replace(" ","."))

.22.3..2..
.3.3.2.3.2
....2..22.
3.3332.2.2
2..2..222.
.3..12.2.2
1.1.2..23.
2.1...2112
1...3.3.2.
3.13.1...3


In [4]:
#data = """\
#33...3.
#..2....
#...21.3
#23...02
#0.03...
#....0..
#.3...33"""
def solve(d):
    pz = "\n".join(["".join(r) for r in d]).replace(" ",".").split()
    nw, nh = len(pz[0]), len(pz)
    M = (nw + 1) * (nh + 1) - 1
    r2 = range(2)
    m = LpProblem()
    vh = np.array(addbinvars(nh+1, nw, 2)) # 0:to left, 1:to right (1)
    vv = np.array(addbinvars(nh, nw+1, 2)) # 0:to down, 1:to up (2)
    vz = np.array(addvars(nh+1, nw+1)) # (3)
    def dirs(i, j, k):
        return ([vh[i,j-l,k^l] for l in r2 if 0 <= j-l < nw] +
                [vv[i-l,j,k^l] for l in r2 if 0 <= i-l < nh])
    for i, j in product(range(nh), range(nw)):
        if pz[i][j].isdigit():
            m += lpSum(vh[i+k,j,l] + vv[i,j+k,l] for l in r2 for k in r2) == int(pz[i][j]) # (4)
            if pz[i][j] == '3':
                fy, fx = i, j
    m += vz[fy,fx] == 0 # (7)
    for i, j in product(range(nh+1), range(nw)):
        m += lpSum(vh[i,j]) <= 1 # (5)
        m += vz[i,j] + 1 <= vz[i,j+1] + M * (1 - vh[i,j,0]) # (6)
        if (i, j) != (fy, fx):
            m += vz[i,j+1] + 1 <= vz[i,j] + M * (1 - vh[i,j,1]) # (6)
    for i, j in product(range(nh), range(nw+1)):
        m += lpSum(vv[i,j]) <= 1 # (5)
        m += vz[i,j] + 1 <= vz[i+1,j] + M * (1 - vv[i,j,0]) # (6)
        if (i, j) != (fy, fx):
            m += vz[i+1,j] + 1 <= vz[i,j] + M * (1 - vv[i,j,1]) # (6)
    for i, j in product(range(nh+1), range(nw+1)):
        m += vz[i,j] <= M # (8)
        din = dirs(i, j, 1)
        dout = dirs(i, j, 0)
        m += lpSum(din) <= 1 # (9)
        m += lpSum(din) == lpSum(dout) # (10)
    %time m.solve()
    rh = np.vectorize(value)(vh.sum(2))
    rv = np.vectorize(value)(vv.sum(2))
    #print(rv.shape) #その位置とその位置 + 1
    #print(rh.shape)#〃
    padv = np.vstack([rv,np.zeros((1,rv.shape[1]))])
    padh = np.hstack([rh,np.zeros((rh.shape[0],1))]) 
    return (pz,np.stack([padv,padh]))
pz,rvh = solve(data[0])

Wall time: 4.29 s


### デモ

In [6]:
_,nh,nw = rvh.shape
for i in range(nh):
    print(' ', end='')
    for j in range(nw):
        print('- ' if rvh[1,i,j] else '  ', end='')
    print()
    if i == nh - 1: break
    for j in range(nw):
        print('|' if rvh[0,i,j] else ' ', end='')
        if j < nh - 1:
            print(pz[i][j], end='')
    print()

 -   - - -   - - - -   
|.|2|2 . 3|.|. 2 . .|
       - -     - - -   
|.|3|.|3 . 2|.|3 . 2 
   -   - - -   - - -   
|. . . . 2 . . 2 2 .|
 -   -   -   - - - -   
 3|.|3|3|3|2|. 2 . 2 
 -     -       - - -   
|2 .|. 2 .|.|2|2 2 .|
   -   - -       - -   
|.|3 .|. 1 2|.|2|. 2 
   -   -   -       -   
|1 .|1 .|2|. .|2|3|.|
   -   -   - -   -     
|2|. 1|. . . 2 1 1 2|
   -   - -   - -   -   
|1 .|. . 3|.|3 .|2|. 
   -   - -   -     -   
|3|. 1|3 . 1 .|.|. 3|
 -     - - - -   - -   


In [ ]:
ans = [solve(data[i]) for i in range(data.shape[0])]

### 保存

In [17]:
ans_vh = [vh for _,vh in ans]
ans_vh = np.array(ans_vh)
np.savez("./dataset",data,ans_vh)